In [1]:
import pandas as pd

cols = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship'
        ,'race','sex','capital-gain','capital-loss','hours-per-week','native-country','Salary']
adult_data = pd.read_csv('https://raw.githubusercontent.com/zekelabs/data-science-complete-tutorial/master/Data/adult.data.txt', names=cols)

In [2]:
adult_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [56]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

In [63]:
from sklearn.naive_bayes import MultinomialNB

In [19]:
cat_cols = adult_data.select_dtypes(include=['object']).columns

In [24]:
cat_cols = list(cat_cols)
cat_cols.remove('Salary')

In [20]:
num_cols = adult_data.select_dtypes(exclude=['object']).columns

In [27]:
cat_feature_selector = SelectKBest(k=5, score_func=mutual_info_classif)

In [38]:
df = pd.DataFrame()

for col in cat_cols:
    le = LabelEncoder()
    df[col]  = le.fit_transform(adult_data[col])
    
X1 = cat_feature_selector.fit_transform(df, adult_data.Salary)

In [31]:
num_cols = num_cols.to_list()

In [36]:
num_feature_selector = SelectKBest(k=3, score_func=f_classif)

In [39]:
X2 = num_feature_selector.fit_transform(df,adult_data.Salary)

In [40]:
import numpy as np

In [42]:
X = np.hstack([X1,X2])

In [47]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [48]:
dt = DecisionTreeClassifier()
lr = LogisticRegression()

In [49]:
dt.fit(X, adult_data.Salary)
lr.fit(X, adult_data.Salary)

/home/awantik/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [46]:
dt.score(X, adult_data.Salary)

0.8356930069715304

In [50]:
lr.score(X, adult_data.Salary)

0.7547986855440558

In [55]:
pipeline_num = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('num_feature_selector',num_feature_selector),
    ('scaling',MinMaxScaler())
])

In [75]:
pipeline_cat = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    #('cat_feature_selector', cat_feature_selector),
    ('encoding', OneHotEncoder(handle_unknown='ignore')),
    ('cat_feature_selector', cat_feature_selector)
])

In [76]:
from sklearn.compose import ColumnTransformer

In [77]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', pipeline_num, num_cols),
        ('cat', pipeline_cat, cat_cols)])

In [78]:
pipeline = Pipeline(steps=[('preprocessor',preprocessor),
                ('classifier', MultinomialNB())])

In [79]:
adult_data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'Salary'],
      dtype='object')

In [95]:
pipeline.fit(adult_data, adult_data.Salary)

Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbo..., 'native-country'])])), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [81]:
adult_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
Salary            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [94]:
adult_data[adult_data['native-country'].isnull()].size

8745

In [85]:
import numpy as np
for col in cat_cols:
    adult_data[col] = adult_data[col].str.strip().replace('?',np.nan)

In [86]:
adult_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         30725 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        30718 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    31978 non-null object
Salary            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [111]:
pipeline.steps[0][1].transformers

[('num', Pipeline(memory=None,
       steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
         strategy='median', verbose=0)), ('num_feature_selector', SelectKBest(k=3, score_func=<function f_classif at 0x7f070efa2ae8>)), ('scaling', MinMaxScaler(copy=True, feature_range=(0, 1)))]), ['age',
   'fnlwgt',
   'education-num',
   'capital-gain',
   'capital-loss',
   'hours-per-week']),
 ('cat', Pipeline(memory=None,
       steps=[('imputer', SimpleImputer(copy=True, fill_value='missing', missing_values=nan,
         strategy='constant', verbose=0)), ('encoding', OneHotEncoder(categorical_features=None, categories=None,
         dtype=<class 'numpy.float64'>, handle_unknown='ignore',
         n_values=None, sparse=True)), ('cat_feature_selector', SelectKBest(k=5, score_func=<function mutual_info_classif at 0x7f070f071ea0>))]), ['workclass',
   'education',
   'marital-status',
   'occupation',
   'relationship',
   'race',
   'sex',
   'native-country'])]

In [122]:
grid_params = {'preprocessor__num__num_feature_selector__k':[5,4,6],
               'preprocessor__num__num_feature_selector__score_func':[chi2,mutual_info_classif]}

In [123]:
from sklearn.model_selection import GridSearchCV

In [124]:
gs = GridSearchCV(pipeline, grid_params,cv=5)

In [125]:
gs.fit(adult_data, adult_data.Salary)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbo..., 'native-country'])])), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'preprocessor__num__num_feature_selector__k': [5, 4, 6], 'preprocessor__num__num_feature_selector__score_func': [<function chi2 at 0x7f070efa2bf8>, <function mutual_info_classif at 0x7f070f071ea0>]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [126]:
gs.best_params_

{'preprocessor__num__num_feature_selector__k': 4,
 'preprocessor__num__num_feature_selector__score_func': <function sklearn.feature_selection.univariate_selection.chi2(X, y)>}

In [127]:
gs.best_score_

0.7197260526396609